# 构建文本生成应用

在本课程中你已经了解到一些核心概念，比如提示词（prompts），甚至还有一个叫做“提示词工程”的专门领域。很多你能接触到的工具，比如 ChatGPT、Office 365、Microsoft Power Platform 等，都支持你通过提示词来完成任务。

如果你想在自己的应用中加入类似体验，就需要理解提示词、补全（completions）等概念，并选择合适的库来实现。本章会带你学习这些内容。

## 介绍

在本章中，你将会：

- 了解 openai 库及其核心概念。
- 使用 openai 构建一个文本生成应用。
- 学习如何利用提示词、温度（temperature）、tokens 等概念来构建文本生成应用。

## 学习目标

本课结束后，你将能够：

- 解释什么是文本生成应用。
- 使用 openai 构建文本生成应用。
- 配置应用使用更多或更少的 tokens，并调整温度参数，让输出更加多样化。

## 什么是文本生成应用？

通常你开发的应用会有类似下面的界面：

- 命令式。控制台应用就是典型的命令式应用，你输入命令，它就执行任务。例如，`git` 就是一个命令式应用。
- 用户界面（UI）。有些应用有图形界面（GUI），你可以点击按钮、输入文本、选择选项等。

### 控制台和 UI 应用的局限

和命令式应用相比：

- **功能有限**。你不能随便输入任何命令，只能用应用支持的那些。
- **语言受限**。有些应用支持多种语言，但默认还是针对某一种语言开发的，虽然可以扩展语言支持。

### 文本生成应用的优势

那么文本生成应用有什么不同呢？

在文本生成应用中，你有更大的灵活性，不再受限于固定命令或特定输入语言。你可以直接用自然语言和应用交互。另一个好处是，你用的数据源已经在海量信息上训练过，而传统应用可能只依赖数据库里的内容。

### 文本生成应用能做什么？

你可以开发很多类型的应用，比如：

- **聊天机器人**。可以回答各种问题，比如关于公司和产品的信息，非常适合用文本生成。
- **助手**。大语言模型（LLM）擅长总结文本、提取信息、生成简历等内容。
- **代码助手**。根据你用的语言模型，可以开发帮助你写代码的助手。例如，GitHub Copilot 和 ChatGPT 都能帮你写代码。

## 如何开始？

你需要找到一种方式和 LLM 集成，通常有两种方法：

- 使用 API。你构造 web 请求，发送提示词，获得生成的文本。
- 使用库。库可以封装 API 调用，让使用更简单。

## 库/SDK

有一些知名的库可以用来操作 LLM，比如：

- **openai**，这个库可以方便地连接模型并发送提示词。

还有一些更高层的库，比如：

- **Langchain**。Langchain 很有名，支持 Python。
- **Semantic Kernel**。Semantic Kernel 是微软开发的库，支持 C#、Python 和 Java。

## 第一个 openai 应用

下面我们来看看如何构建第一个应用，需要哪些库，具体步骤是什么。

### 安装 openai

  > [!NOTE] 如果你在 Codespaces 或 Devcontainer 里运行本笔记本，这一步可以跳过

市面上有很多库可以和 OpenAI 或 Azure OpenAI 交互。你可以用多种编程语言，比如 C#、Python、JavaScript、Java 等。  
我们这里选择用 `openai` 的 Python 库，所以用 `pip` 来安装。

```bash
pip install openai
```

如果你不是在 Codespaces 或 Dev Container 里运行，还需要在你的电脑上安装 [Python](https://www.python.org/)。

### 创建资源

如果你还没做过，需要完成以下步骤：

- 在 Azure 上创建账号 <https://azure.microsoft.com/free/>。
- 获取 Azure OpenAI 的访问权限。访问 <https://learn.microsoft.com/azure/ai-services/openai/overview#how-do-i-get-access-to-azure-openai> 并申请权限。

  > [!NOTE]
  > 截至目前，Azure OpenAI 需要申请访问权限。

- 创建 Azure OpenAI 服务资源。可以参考这个指南 [创建资源](https://learn.microsoft.com/azure/ai-services/openai/how-to/create-resource?pivots=web-portal&WT.mc_id=academic-105485-koreyst)。

### 找到 API 密钥和终端地址

现在你需要告诉 `openai` 库用哪个 API 密钥。你可以在 Azure OpenAI 资源的“Keys and Endpoint”部分找到你的 API 密钥，复制“Key 1”的值。

  ![Azure Portal 中的 Keys and Endpoint 资源界面](https://learn.microsoft.com/azure/ai-services/openai/media/quickstarts/endpoint.png?WT.mc_id=academic-105485-koreyst)

有了这些信息后，我们来设置库的相关参数。

> [!NOTE]
> 建议把 API 密钥和代码分开存放。可以用环境变量来实现。
> - 在 .env 文件里设置环境变量 `AZURE_OPENAI_API_KEY`，值为你的 API 密钥。如果你已经完成了本课程前面的练习，就已经设置好了。

### 配置 Azure

如果你用的是 Azure OpenAI，可以这样配置：

```python
client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  
  api_version = "2023-10-01-preview"
  azure_endpoint = os.environ('AZURE_OPENAI_ENDPOINT')
  )

deployment = os.environ['AZURE_OPENAI_DEPLOYMENT']
```

上面我们设置了以下内容：

- `api_key`，就是你在 Azure Portal 里找到的 API 密钥。
- `api_version`，你想用的 API 版本。目前最新的是 `2023-10-01-preview`。
- `azure_endpoint`，API 的终端地址。在 Azure Portal 里和 API 密钥一起显示。

> [!NOTE]
> `os.environ` 是读取环境变量的函数。你可以用它来读取像 `AZURE_OPENAI_API_KEY` 和 `AZURE_OPENAI_ENDPOINT` 这样的环境变量。

## 生成文本

生成文本的方法是用 `chat.completion` 类。下面是一个例子：

```python
prompt = "Complete the following: Once upon a time there was a"

completion = client.chat.completions.create(model=deployment, messages=[{"role": "user", "content": prompt}])
print(completion.choices[0].message.content)
```

在上面的代码里，我们创建了一个补全对象，传入要用的模型和提示词，然后打印生成的文本。

### 聊天补全

到目前为止，你已经看到我们用 `Completion` 来生成文本。其实还有一个叫做 `ChatCompletion` 的类，更适合做聊天机器人。下面是一个使用方法的例子：

```python
client = AzureOpenAI(
  azure_endpoint = os.environ('AZURE_OPENAI_ENDPOINT'), 
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  
  api_version = "2023-05-15"
  )

deployment = os.environ['AZURE_OPENAI_DEPLOYMENT']

completion = client.chat.completions.create(model=deployment, messages=[{"role": "user", "content": "Hello world"}])
print(completion.choices[0].message.content)
```

后续章节会详细介绍这个功能。

## 练习 - 你的第一个文本生成应用

现在我们已经学会了如何设置和配置 Azure OpenAI 服务，是时候来构建你的第一个文本生成应用了。按照以下步骤来完成你的应用：


1. 创建虚拟环境并安装 openai：

  > [!NOTE] 如果你在 Codespaces 或 Devcontainer 中运行此笔记本，则无需执行此步骤


In [ ]:
# Create virtual environment
! python -m venv venv
# Activate virtual environment
! source venv/bin/activate
# Install openai package
! pip install openai

> [!NOTE]
> 如果你使用的是 Windows，请输入 `venv\Scripts\activate`，而不是 `source venv/bin/activate`。

> [!NOTE]
> 要找到你的 Azure OpenAI 密钥，请访问 https://portal.azure.com/，搜索 `Open AI`，选择 `Open AI resource`，然后点击 `Keys and Endpoint`，复制 `Key 1` 的值。


1. 创建一个 *app.py* 文件，并输入以下代码：


In [ ]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"], 
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  
  api_version = "2023-10-01-preview"
  )

deployment = os.environ['AZURE_OPENAI_DEPLOYMENT']

# add your completion code
prompt = "Complete the following: Once upon a time there was a"
messages = [{"role": "user", "content": prompt}]  

# make completion
completion = client.chat.completions.create(model=deployment, messages=messages)

# print response
print(completion.choices[0].message.content)

```output
     very unhappy _____.

    Once upon a time there was a very unhappy mermaid.
    ```


## 不同类型的提示词，用于不同的场景

现在你已经了解了如何用提示词生成文本。你甚至已经有了一个可以修改和调整的程序，用来生成不同类型的文本。

提示词可以用于各种任务。例如：

- **生成某种类型的文本**。比如，你可以生成一首诗、出一些测验题等等。
- **查找信息**。你可以用提示词来查找信息，比如下面这个例子：“在网页开发中，CORS是什么意思？”。
- **生成代码**。你可以用提示词来生成代码，比如开发一个用于验证邮箱的正则表达式，甚至可以生成一个完整的程序，比如一个网页应用。

## 更实用的场景：食谱生成器

假设你家里有一些食材，想做点什么吃的。你需要一个食谱。找食谱的方法可以用搜索引擎，也可以用大语言模型来帮忙。

你可以这样写一个提示词：

> “请给我展示5个用以下食材做的菜谱：鸡肉、土豆和胡萝卜。每个菜谱请列出所有用到的食材。”

根据上面的提示词，你可能会得到类似这样的回复：

```output
1. Roasted Chicken and Vegetables: 
Ingredients: 
- 4 chicken thighs
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 2 tablespoons olive oil
- 2 cloves garlic, minced
- 1 teaspoon dried thyme
- 1 teaspoon dried oregano
- Salt and pepper, to taste

2. Chicken and Potato Stew: 
Ingredients: 
- 2 tablespoons olive oil
- 1 onion, diced
- 2 cloves garlic, minced
- 2 chicken breasts, cut into cubes
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 1 teaspoon dried oregano
- 1 teaspoon dried thyme
- 1 cup chicken broth
- Salt and pepper, to taste

3. Chicken and Potato Bake: 
Ingredients: 
- 2 tablespoons olive oil
- 2 chicken breasts, cut into cubes
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 1 onion, diced
- 2 cloves garlic, minced
- 1 teaspoon dried oregano
- 1 teaspoon dried thyme
- 1 cup chicken broth
- Salt and pepper, to taste

4. Chicken and Potato Soup: 
Ingredients: 
- 2 tablespoons olive oil
- 1 onion, diced
- 2 cloves garlic, minced
- 2 chicken breasts, cut into cubes
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 1 teaspoon dried oregano
- 1 teaspoon dried thyme
- 4 cups chicken broth
- Salt and pepper, to taste

5. Chicken and Potato Hash: 
Ingredients: 
- 2 tablespoons olive oil
- 2 chicken breasts, cut into cubes
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 1 onion, diced
- 2 cloves garlic, minced
- 1 teaspoon dried oregano
```

这个结果很棒，我知道该做什么了。这个时候，可能有一些可以改进的地方：

- 把我不喜欢或者过敏的食材过滤掉。
- 如果家里没有所有食材，可以生成一个购物清单。

针对上面的情况，我们可以加一个补充提示词：

> “请去掉含有大蒜的菜谱，因为我对大蒜过敏，并用其他食材替换。同时，请为这些菜谱生成一个购物清单，考虑到我家里已经有鸡肉、土豆和胡萝卜。”

现在你会得到一个新的结果，比如：

```output
1. Roasted Chicken and Vegetables: 
Ingredients: 
- 4 chicken thighs
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 2 tablespoons olive oil
- 1 teaspoon dried thyme
- 1 teaspoon dried oregano
- Salt and pepper, to taste

2. Chicken and Potato Stew: 
Ingredients: 
- 2 tablespoons olive oil
- 1 onion, diced
- 2 chicken breasts, cut into cubes
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 1 teaspoon dried oregano
- 1 teaspoon dried thyme
- 1 cup chicken broth
- Salt and pepper, to taste

3. Chicken and Potato Bake: 
Ingredients: 
- 2 tablespoons olive oil
- 2 chicken breasts, cut into cubes
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 1 onion, diced
- 1 teaspoon dried oregano
- 1 teaspoon dried thyme
- 1 cup chicken broth
- Salt and pepper, to taste

4. Chicken and Potato Soup: 
Ingredients: 
- 2 tablespoons olive oil
- 1 onion, diced
- 2 chicken breasts, cut into cubes
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 1 teaspoon dried oregano
- 1 teaspoon dried thyme
- 4 cups chicken broth
- Salt and pepper, to taste

5. Chicken and Potato Hash: 
Ingredients: 
- 2 tablespoons olive oil
- 2 chicken breasts, cut into cubes
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 1 onion, diced
- 1 teaspoon dried oregano

Shopping List: 
- Olive oil
- Onion
- Thyme
- Oregano
- Salt
- Pepper
```

这就是你的五个菜谱，没有提到大蒜，还根据你家里已有的食材生成了购物清单。


## 练习 - 构建一个食谱生成器

现在我们已经演示了一个场景，接下来让我们编写代码来实现这个场景。请按照以下步骤操作：

1. 以现有的 *app.py* 文件为起点
1. 找到 `prompt` 变量，并将其代码更改为以下内容：


In [ ]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

# load environment variables from .env file
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"], 
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  
  api_version = "2023-10-01-preview"
  )

deployment = os.environ['AZURE_OPENAI_DEPLOYMENT']

prompt = "Show me 5 recipes for a dish with the following ingredients: chicken, potatoes, and carrots. Per recipe, list all the ingredients used"
messages = [{"role": "user", "content": prompt}]  

# make completion
completion = client.chat.completions.create(model=deployment, messages=messages, max_tokens=600)

# print response
print(completion.choices[0].message.content)

如果你现在运行代码，应该会看到类似下面的输出：

```output
-Chicken Stew with Potatoes and Carrots: 3 tablespoons oil, 1 onion, chopped, 2 cloves garlic, minced, 1 carrot, peeled and chopped, 1 potato, peeled and chopped, 1 bay leaf, 1 thyme sprig, 1/2 teaspoon salt, 1/4 teaspoon black pepper, 1 1/2 cups chicken broth, 1/2 cup dry white wine, 2 tablespoons chopped fresh parsley, 2 tablespoons unsalted butter, 1 1/2 pounds boneless, skinless chicken thighs, cut into 1-inch pieces
-Oven-Roasted Chicken with Potatoes and Carrots: 3 tablespoons extra-virgin olive oil, 1 tablespoon Dijon mustard, 1 tablespoon chopped fresh rosemary, 1 tablespoon chopped fresh thyme, 4 cloves garlic, minced, 1 1/2 pounds small red potatoes, quartered, 1 1/2 pounds carrots, quartered lengthwise, 1/2 teaspoon salt, 1/4 teaspoon black pepper, 1 (4-pound) whole chicken
-Chicken, Potato, and Carrot Casserole: cooking spray, 1 large onion, chopped, 2 cloves garlic, minced, 1 carrot, peeled and shredded, 1 potato, peeled and shredded, 1/2 teaspoon dried thyme leaves, 1/4 teaspoon salt, 1/4 teaspoon black pepper, 2 cups fat-free, low-sodium chicken broth, 1 cup frozen peas, 1/4 cup all-purpose flour, 1 cup 2% reduced-fat milk, 1/4 cup grated Parmesan cheese

-One Pot Chicken and Potato Dinner: 2 tablespoons olive oil, 1 pound boneless, skinless chicken thighs, cut into 1-inch pieces, 1 large onion, chopped, 3 cloves garlic, minced, 1 carrot, peeled and chopped, 1 potato, peeled and chopped, 1 bay leaf, 1 thyme sprig, 1/2 teaspoon salt, 1/4 teaspoon black pepper, 2 cups chicken broth, 1/2 cup dry white wine

-Chicken, Potato, and Carrot Curry: 1 tablespoon vegetable oil, 1 large onion, chopped, 2 cloves garlic, minced, 1 carrot, peeled and chopped, 1 potato, peeled and chopped, 1 teaspoon ground coriander, 1 teaspoon ground cumin, 1/2 teaspoon ground turmeric, 1/2 teaspoon ground ginger, 1/4 teaspoon cayenne pepper, 2 cups chicken broth, 1/2 cup dry white wine, 1 (15-ounce) can chickpeas, drained and rinsed, 1/2 cup raisins, 1/2 cup chopped fresh cilantro
```

> NOTE，你的LLM是非确定性的，所以每次运行程序时，结果可能会有所不同。

很好，接下来我们看看如何改进。为了让程序更好用，我们希望代码更加灵活，这样配料和菜谱数量都可以方便地调整和修改。


1. 让我们按以下方式更改代码：


In [ ]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

# load environment variables from .env file
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"], 
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  
  api_version = "2023-10-01-preview"
  )

deployment = os.environ['AZURE_OPENAI_DEPLOYMENT']

no_recipes = input("No of recipes (for example, 5: ")

ingredients = input("List of ingredients (for example, chicken, potatoes, and carrots: ")

# interpolate the number of recipes into the prompt an ingredients
prompt = f"Show me {no_recipes} recipes for a dish with the following ingredients: {ingredients}. Per recipe, list all the ingredients used"
messages = [{"role": "user", "content": prompt}]  

# make completion
completion = client.chat.completions.create(model=deployment, messages=messages, max_tokens=600)

# print response
print(completion.choices[0].message.content)

运行代码进行测试，可能如下所示：

```output
No of recipes (for example, 5: 3
List of ingredients (for example, chicken, potatoes, and carrots: milk,strawberries

-Strawberry milk shake: milk, strawberries, sugar, vanilla extract, ice cubes
-Strawberry shortcake: milk, flour, baking powder, sugar, salt, unsalted butter, strawberries, whipped cream        
-Strawberry milk: milk, strawberries, sugar, vanilla extract
```

### 通过添加筛选和购物清单来改进

我们现在已经有了一个可以生成菜谱的应用，并且它很灵活，因为它依赖于用户输入，无论是菜谱数量还是所用食材。

为了进一步完善它，我们希望添加以下功能：

- **筛选食材**。我们希望能够筛除我们不喜欢或过敏的食材。为实现这个功能，我们可以编辑现有的提示词，在结尾加上筛选条件，如下所示：

    ```python
    filter = input("Filter (for example, vegetarian, vegan, or gluten-free: ")

    prompt = f"Show me {no_recipes} recipes for a dish with the following ingredients: {ingredients}. Per recipe, list all the ingredients used, no {filter}"
    ```

    上面，我们在提示词结尾加上了 `{filter}`，同时也从用户那里获取了筛选条件。

    现在，运行程序的输入示例如下：

    ```output    
    No of recipes (for example, 5: 3
    List of ingredients (for example, chicken, potatoes, and carrots: onion,milk
    Filter (for example, vegetarian, vegan, or gluten-free: no milk

    1. French Onion Soup

    Ingredients:
    
    -1 large onion, sliced
    -3 cups beef broth
    -1 cup milk
    -6 slices french bread
    -1/4 cup shredded Parmesan cheese
    -1 tablespoon butter
    -1 teaspoon dried thyme
    -1/4 teaspoon salt
    -1/4 teaspoon black pepper
    
    Instructions:
    
    1. In a large pot, sauté onions in butter until golden brown.
    2. Add beef broth, milk, thyme, salt, and pepper. Bring to a boil.
    3. Reduce heat and simmer for 10 minutes.
    4. Place french bread slices on soup bowls.
    5. Ladle soup over bread.
    6. Sprinkle with Parmesan cheese.
    
    2. Onion and Potato Soup
    
    Ingredients:
    
    -1 large onion, chopped
    -2 cups potatoes, diced
    -3 cups vegetable broth
    -1 cup milk
    -1/4 teaspoon black pepper
    
    Instructions:
    
    1. In a large pot, sauté onions in butter until golden brown.
    2. Add potatoes, vegetable broth, milk, and pepper. Bring to a boil.
    3. Reduce heat and simmer for 10 minutes.
    4. Serve hot.
    
    3. Creamy Onion Soup
    
    Ingredients:
    
    -1 large onion, chopped
    -3 cups vegetable broth
    -1 cup milk
    -1/4 teaspoon black pepper
    -1/4 cup all-purpose flour
    -1/2 cup shredded Parmesan cheese
    
    Instructions:
    
    1. In a large pot, sauté onions in butter until golden brown.
    2. Add vegetable broth, milk, and pepper. Bring to a boil.
    3. Reduce heat and simmer for 10 minutes.
    4. In a small bowl, whisk together flour and Parmesan cheese until smooth.
    5. Add to soup and simmer for an additional 5 minutes, or until soup has thickened.
    ```

    可以看到，所有含有牛奶的菜谱都被筛除了。但如果你乳糖不耐受，可能还想把含有奶酪的菜谱也筛掉，所以需要明确说明。

    ```python
    
- **Produce a shopping list**. We want to produce a shopping list, considering what we already have at home.

    For this functionality, we could either try to solve everything in one prompt or we could split it up into two prompts. Let's try the latter approach. Here we're suggesting adding an additional prompt, but for that to work, we need to add the result of the former prompt as context to the latter prompt. 

    Locate the part in the code that prints out the result from the first prompt and add the following code below:
    
    ```python
    old_prompt_result = completion.choices[0].text
    prompt = "Produce a shopping list for the generated recipes and please don't include ingredients that I already have."
    
    new_prompt = f"{old_prompt_result} {prompt}"
    messages = [{"role": "user", "content": new_prompt}]
    completion = client.chat.completion.create(model=deployment, messages=messages, max_tokens=1200)
    
    # 打印响应
    print("购物清单:")
    print(completion.choices[0].message.content)
    ```

    Note the following:

    - We're constructing a new prompt by adding the result from the first prompt to the new prompt: 
    
        ```python
        new_prompt = f"{old_prompt_result} {prompt}"
        messages = [{"role": "user", "content": new_prompt}]
        ```

    - We make a new request, but also considering the number of tokens we asked for in the first prompt, so this time we say `max_tokens` is 1200. 

        ```python
        completion = client.chat.completion.create(model=deployment, messages=messages, max_tokens=1200)
        ```  

        Taking this code for a spin, we now arrive at the following output:

        ```output
        No of recipes (for example, 5: 2
        List of ingredients (for example, chicken, potatoes, and carrots: apple,flour
        Filter (for example, vegetarian, vegan, or gluten-free: sugar
        Recipes:
         or milk.
        
        -Apple and flour pancakes: 1 cup flour, 1/2 tsp baking powder, 1/2 tsp baking soda, 1/4 tsp salt, 1 tbsp sugar, 1 egg, 1 cup buttermilk or sour milk, 1/4 cup melted butter, 1 Granny Smith apple, peeled and grated
        -Apple fritters: 1-1/2 cups flour, 1 tsp baking powder, 1/4 tsp salt, 1/4 tsp baking soda, 1/4 tsp nutmeg, 1/4 tsp cinnamon, 1/4 tsp allspice, 1/4 cup sugar, 1/4 cup vegetable shortening, 1/4 cup milk, 1 egg, 2 cups shredded, peeled apples
        Shopping list:
         -Flour, baking powder, baking soda, salt, sugar, egg, buttermilk, butter, apple, nutmeg, cinnamon, allspice 
        ```
        
- **A word on token length**. We should consider how many tokens we need to generate the text we want. Tokens cost money, so where possible, we should try to be economical with the number of tokens we use. For example, can we phrase the prompt so that we can use less tokens?

   To change tokens used, you can use the `max_tokens` parameter. For example, if you want to use 100 tokens, you would do:

    ```python
    completion = client.chat.completion.create(model=deployment, messages=messages, max_tokens=100)
    ```

- **Experimenting with temperature**. Temperature is something we haven't mentioned so far but is an important context for how our program performs. The higher the temperature value the more random the output will be. Conversely the lower the temperature value the more predictable the output will be. Consider whether you want variation in your output or not.

   To alter the temperature, you can use the `temperature` parameter. For example, if you want to use a temperature of 0.5, you would do:

    ```python
    completion = client.chat.completion.create(model=deployment, messages=messages, temperature=0.5)
    ```

   > 注意，温度值越接近 1.0，输出内容就越多样化。



## 练习

本次练习，你可以自由选择要开发的内容。

以下是一些建议：

- 对菜谱生成器应用进行进一步优化。尝试调整 temperature 的值，以及修改提示词，看看能得到什么样的结果。
- 开发一个“学习助手”。这个应用应该能回答某个主题的问题，比如 Python，你可以设置类似“Python 中某个主题是什么？”的问题，或者让它展示某个主题的代码等。
- 历史机器人，让历史变得生动起来，指示机器人扮演某个历史人物，并向它提问有关其生平和时代的问题。

## 参考答案

### 学习助手

- “你是 Python 语言的专家

    请用以下格式为 Python 初学者推荐一节课：

    格式：
    - 概念：
    - 课程简要说明：
    - 代码练习及答案："

上面是一个起始提示词，可以根据你的需求进行调整。

### 历史机器人

你可以使用如下提示词：

- “你是林肯，请用三句话介绍一下你自己，并用林肯当时的语法和用词来回答”
- “你是林肯，请用林肯当时的语法和用词来回答：

   用 300 字介绍你最伟大的成就：”

## 知识检测

temperature 这个参数的作用是什么？

1. 它控制输出的随机性。
1. 它控制回复的长度。
1. 它控制使用的 token 数量。

A: 1

存储 API 密钥等敏感信息的好方法是什么？

1. 写在代码里。
1. 存在文件里。
1. 存在环境变量里。

A: 3，因为环境变量不会写在代码里，可以在代码中加载。



---

**免责声明**：  
本文件由 AI 翻译服务 [Co-op Translator](https://github.com/Azure/co-op-translator) 翻译。我们力求准确，但请注意，自动翻译可能包含错误或不准确之处。原始语言的文件应被视为权威来源。对于关键信息，建议使用专业人工翻译。因使用本翻译而产生的任何误解或曲解，我们概不负责。
